In [5]:
import json
import os
from tqdm import tqdm
import shutil

In [2]:
# Reading JSON with labels for each image
with open("data/bdd100k/bdd100k_labels_images_train.json", "r") as file:
    train_data = json.load(file)

In [6]:
# Reading JSON with labels for each image
with open("data/bdd100k/bdd100k_labels_images_val.json", "r") as file:
    val_data = json.load(file)

In [4]:
train_label_names = [label["name"] for label in train_data]
train_image_names = os.listdir("data/bdd100k/train/images")

In [5]:
print(f"Number of train labels: {len(train_label_names)}")
print(f"Number of train images: {len(train_image_names)}")

Number of train labels: 69863
Number of train images: 69863


In [6]:
# Removing images without labels
remove_counter = 0
for image in tqdm(train_image_names):
    if image not in train_label_names:
        os.remove(f"data/bdd100k/train/images/{image}")
        remove_counter += 1
print(f"Removed {remove_counter} images")

100%|██████████| 69863/69863 [00:36<00:00, 1903.16it/s] 

Removed 0 images


In [7]:
val_label_names = [label["name"] for label in val_data]
val_image_names = os.listdir("data/bdd100k/val/images/")

In [8]:
print(f"Number of train labels: {len(val_label_names)}")
print(f"Number of train images: {len(val_image_names)}")

Number of train labels: 10000
Number of train images: 10001


### Parsing JSON

In [7]:
categories = {
    "car": 0,
    "bus": 0,
    "truck": 0,
    "person": 1,
    "traffic sign": 2,
    "traffic light": 3,
}

In [8]:
image_width = 1280
image_heigth = 720

In [9]:
files = os.listdir("data/bdd100k/val/images/")

In [10]:
# Saving Images with Clear weather
def get_clear_image_data(image, img_path, label_path):
    if image["name"] in files:
        if image["attributes"]["weather"] == "clear" and image["attributes"]["timeofday"] == "daytime":
            shutil.copy(f"data/bdd100k/val/images/{image["name"]}", img_path)
            for label in image["labels"]:
                if label["category"] in categories:
                    category = categories[label["category"]]
                    x_center = (label["box2d"]["x1"] + label["box2d"]["x2"]) / (2 * image_width)
                    y_center = (label["box2d"]["y1"] + label["box2d"]["y2"]) / (2 * image_heigth)
                    width = (label["box2d"]["x2"] - label["box2d"]["x1"]) / (image_width)
                    height = (label["box2d"]["y2"] - label["box2d"]["y1"]) / (image_heigth)
                    with open(f"{label_path}/{image["name"].replace(".jpg", "")}.txt", "a") as file:
                        file.write(f"{category} {x_center} {y_center} {width} {height}\n")
    pass

In [36]:
for img in tqdm(train_data):
    get_clear_image_data(img, "data/bdd100k_clear/train/images/", "data/bdd100k_clear/train/labels")

100%|██████████| 69863/69863 [01:05<00:00, 1074.28it/s]


In [11]:
for img in tqdm(val_data):
    get_clear_image_data(img, "data/bdd100k_clear/val/images/", "data/bdd100k_clear/val/labels")

100%|██████████| 10000/10000 [00:06<00:00, 1513.58it/s]


### YOLO training

In [ ]:
from ultralytics import YOLO
import os

In [3]:
len(os.listdir("data/bdd100k_clear/train/images/"))

27027

In [13]:
model = YOLO("models/yolo11m.pt")

In [14]:
results = model.train(
    data="dataset.yaml",
    imgsz=640,
    epochs=50,
    batch=8,
    name="clear_yolo11m_50e_640imgz",
)

New https://pypi.org/project/ultralytics/8.3.131 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.122  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: task=detect, mode=train, model=models/yolo11m.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=clear_yolo11m_50e_640imgz2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

train: Scanning C:\Users\aidar\DL\data\bdd100k_clear\train\labels.cache... 12454 images, 14573 backgrounds, 0 corrupt: 100%|██████████| 27027/27027 [00:00<?, ?it/s]

train: C:\Users\aidar\DL\data\bdd100k_clear\train\images\75055858-7d04a650.jpg: 1 duplicate labels removed


val: Fast image access  (ping: 0.00.0 ms, read: 6.01.4 MB/s, size: 67.9 KB)


val: Scanning C:\Users\aidar\DL\data\bdd100k_clear\val\labels... 1764 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1764/1764 [00:01<00:00, 933.93it/s] 

...

      50/50      7.88G      1.087     0.7607     0.9073         15        640: 100%|██████████| 3379/3379 [05:51<00:00,  9.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:09<00:00, 12.15it/s]

                   all       1764      33611      0.717      0.539      0.616      0.336



50 epochs completed in 5.169 hours.
Optimizer stripped from runs\detect\clear_yolo11m_50e_640imgz2\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\clear_yolo11m_50e_640imgz2\weights\best.pt, 40.5MB

Validating runs\detect\clear_yolo11m_50e_640imgz2\weights\best.pt...
Ultralytics 8.3.122  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
YOLO11m summary (fused): 125 layers, 20,033,116 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:11<00:00,  9.72it/s]


                   all       1764      33611      0.718       0.54      0.617      0.336
                   car       1753      21035      0.806       0.68      0.773      0.496
                person        628       2450      0.685      0.472      0.553      0.283
          traffic sign       1461       6339      0.678      0.511      0.577      0.314
         traffic light        817       3787      0.704      0.496      0.564      0.251
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\clear_yolo11m_50e_640imgz2
